# 03 - Simple logistic regression

## Data

Source of data: mlbench (version 2.1-3.1)
PimaIndiansDiabetes: Pima Indians Diabetes Databas

https://cran.r-project.org/web/packages/mlbench/index.html
e

Data set PID2.csv

In [1]:
library(readr)
PID2 <- read_csv("data/PID2.csv",
                 show_col_types = FALSE)
PID2$diabetes_n <- ifelse(PID2$diabetes == "neg", 0, 1)
head(PID2)


pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes,diabetes_n
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,89,66,23,94,28.1,0.167,21,neg,0
0,137,40,35,168,43.1,2.288,33,pos,1
3,78,50,32,88,31.0,0.248,26,pos,1
2,197,70,45,543,30.5,0.158,53,pos,1
1,189,60,23,846,30.1,0.398,59,pos,1
5,166,72,19,175,25.8,0.587,51,pos,1


## Question

Does the glucose level predict diabetes?

Model: log[p(X) / (1-p(X))] = ß0 + ß1X



H0: No significant relationship between glucose and diabetes, b1 = zero.

H1: Significant relationship between glucose and diabetes, ß1 <> zero.

## SAS program snippet

The following SAS code will be executed.

The option descending reverses the order of the levels in the dependent variable.

## Results

The output is divided into blocks to explain it and to reproduce it afterwards in the different languages.

### Block 1
![Block 1](img_screenshots/block_1.png)

Row 1 refers to the dataset which was used in this procedure.

Row 2 gives the response variable or dependent variable for the logistic regression.

Row 3 gives the number of response levels equal to the available levels of the dependent variable in the dataset.

Row 4 names the type of model. In this case it is a logistic regression or binary logit as stated here.

Row 5 gives the name of the optimization technique which was used. Here is a source for differences between the statistical programs.
In SAS, the default method is Fisher’s scoring method.
In R, the glm documentation mentions iteratively reweighted least squares (IWLS) as the method.
In Stata, it is the Newton-Raphson algorithm. 
These are the three main methods.

You have to look into the small print in the description of the method.

### R chunk for reproduction

Data set will be part of the result of the summary() function.

The response variable is given in the formula.

The number of levels can be obtained with the table() function. SAS handles character values, R needs numeric values for the response variable.

The summary() function also provides the information that Fisher Scoring was used.



In [2]:
library(broom) 
my_logistic <- glm(diabetes_n ~ glucose, data = PID2, family = binomial)

summary(my_logistic)

# Number of response levels
table("diabetes" = PID2$diabetes, "diabetes_n" = PID2$diabetes_n)



Call:
glm(formula = diabetes_n ~ glucose, family = binomial, data = PID2)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -6.095521   0.629787  -9.679   <2e-16 ***
glucose      0.042421   0.004761   8.911   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 498.10  on 391  degrees of freedom
Residual deviance: 386.67  on 390  degrees of freedom
AIC: 390.67

Number of Fisher Scoring iterations: 4


        diabetes_n
diabetes   0   1
     neg 262   0
     pos   0 130

### Block 2
![Block 2](img_screenshots/block_2.png)

The number of observations used might be less than the number of observations read.
SAS performs a listwise deletion (complete case analysis) if missing values are present.

### R chunk for reproduction

In [3]:
## Number of observations read
nrow(PID2)
## Number of observations used
nobs(my_logistic)

[1] 392

[1] 392

### Block 3
![Block 3](img_screenshots/block_3.png)

The levels and the frequencies for the dependent variable are provided here.

It is also stated which probability is modeled here. The order was reversed here with the descending option in the proc logistic statement.

By default SAS models the 0 while other statistical programs model the 1. 
Categorical levels would be sorted in alphabetical order and the first level would be modeled.

### R chunk for reproduction

In [4]:
suppressPackageStartupMessages(library(tidyverse))
PID2 %>%
group_by(diabetes, diabetes_n) %>%
count(name = "Total Frequency")

diabetes,diabetes_n,Total Frequency
<chr>,<dbl>,<int>
neg,0,262
pos,1,130


### Block 4
![Block 4](img_screenshots/block_4.png)

The important information that the model converged can be found here.

The model fit status is described by 
-  AIC (Akaike Information Criterion): Smaller is better.
-  SC (Schwarz Criterion): Smaller is better.
-  -2 Log L (negative two times the log-likelihood)



### R chunk for reproduction

In [5]:
glance(my_logistic)

null.deviance,df.null,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
498.0978,391,-193.333,390.666,398.6085,386.666,390,392


null.deviance and df.null refer to the intercept only model.

BIC (Bayes information criterion) is also SC (Schwarz criterion).

you can run an intercept model with the following code.

In [6]:
my_logistic_intercept <- glm(diabetes_n ~ 1, data = PID2, family = binomial)
glance(my_logistic_intercept)

null.deviance,df.null,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
498.0978,391,-249.0489,500.0978,504.0691,498.0978,391,392


The values are similar to the SAS output.

### Block 5
![Block 5](img_screenshots/block_5.png)

These global tests test the null hypothesis that all regression coefficents are zero.

The tests are different chi-square tests.


### R chunk for reproduction

In [7]:
# z-test
# Wald’s chi-squared statistic for the hypothesis that the coefficient of glucose
# is zero, or equivalently that the odds-ratio is one
m0 <- glm(diabetes_n ~ 1, data = PID2, family = binomial)
m1 <- glm(diabetes_n ~ glucose, data = PID2, family = binomial)

b <- coef(m1)
se <- sqrt(diag(vcov(m1)))
(b[2]/se[2])^2

# likeli ratio test
#-2*(logLik(m0) - logLik(m1))
-2*(logLik(m0) - logLik(m1))


glucose 
79.40245

'log Lik.' 111.4318 (df=1)

### Block 6
![Block 6](img_screenshots/block_6.png)

Column 1 "Parameter" lists the intercept and the parameter in the model.

Column 2 "DF" gives the degrees of freedom for every parameter.

Column 3 "Estimate" lists the logit regression estimates for every parameter given that the other parameter are held constant. 

$log(p / (1 - p)) = -6.10 + 0.04 * glucose$ with p as the probability for diabetes.

Column 4 "Standard Error" gives the standard errors of the individual regression coefficients.

Column 5 "Wald Chi-Square" tests the null hypothesis that the regression coefficient is zero given that the other predictors are in the model.

Column 6 "Pr > ChiSq" gives the p-value for the Wald Chi-Square statistic.

### R chunk for reproduction

In [8]:
tidy(my_logistic)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-6.09552139,0.629787038,-9.678703,3.713993e-22
glucose,0.04242099,0.004760623,8.910805,5.066328e-19


*term* represents the *Parameter* in the SAS output.

Degrees of freedom are missing.

The estimate and standard error results are similar.

The statistic is a z test in R and a Wald chi-square test in SAS.

The p-value results from this test.


In [9]:
summary(my_logistic)


Call:
glm(formula = diabetes_n ~ glucose, family = binomial, data = PID2)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -6.095521   0.629787  -9.679   <2e-16 ***
glucose      0.042421   0.004761   8.911   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 498.10  on 391  degrees of freedom
Residual deviance: 386.67  on 390  degrees of freedom
AIC: 390.67

Number of Fisher Scoring iterations: 4


### Block 7
![Block 7](img_screenshots/block_7.png)

Column 1 "Effect" lists the variables which are interpreted by the point estimate.

Column 2 "Point Estimate" is interpreted as an odds ratio. 
One unit change in the independent variable changes the probability for the modelled event by the estimated value.

Column 3 and 4 give the confidence interval for the odds ratio.

### R chunk for reproduction

Odds ratio estimate are calculated by exponentiation of the estimates.

Confidence intervals are returned by the confint() function.

In [10]:
exp(my_logistic$coeff)
suppressMessages(exp(confint(my_logistic)))

(Intercept)     glucose 
0.002252935 1.043333616

,2.5 %,97.5 %
(Intercept),0.0006194309,0.007358113
glucose,1.0340267589,1.053554337


### Block 8
![Block 8](img_screenshots/block_8.png)

These parameter describe the association between the predicted probabilities and observed responses.

For details for calculation see https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.4/statug/statug_logistic_details12.htm

### R chunk for reproduction

The information of block 8 are not directly provided in base R.



In [11]:
library(survival)
concordance(my_logistic)

Call:
concordance.lm(object = my_logistic)

n= 392 
Concordance= 0.8058 se= 0.02356
concordant discordant     tied.x     tied.y    tied.xy 
     27332       6503        225      42049        527 

In [12]:
# Sommer's D is 2C-1
2 * 0.8058 - 1
# Goodman and Kruskal's Gamma (or simple Gamma)
# (#concordant - #discordant) / (#concordant + #discordant)
(27332 - 6503) / (27332 + 6503)
# https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient 
# Tau
# 1 - 2*(number of discordant pairs) / (number of pairs)
# 1 - 2 * 6503 / (n * (n - 1) / 2)
392 * 391 / 2
1 - 2 * 6503 / (392 * (392 - 1) / 2)
# Tau a
# (#concordant - # discordant) / n
(27332 - 6503) / (392*391 / 2)
# Tau c below, but what is c?
# 2 * (nc - nd) / (n^2*(m-1)/m)
2 * (27332 - 6503) / (392**2 * (2 - 1) / 2)

[1] 0.6116

[1] 0.6156051

[1] 76636

[1] 0.8302886

[1] 0.2717913

[1] 0.542196

The others could be calculated by hand according to the formula mentioned in the previous block 8.

In [13]:
library(yardstick)
obs <- as.integer(PID2$diabetes_n)
pred <- as.integer(round(predict(my_logistic, type = "response")))
str(obs)
str(pred)
df1 <- table(obs, pred)


Attache Paket: 'yardstick'


Das folgende Objekt ist maskiert 'package:readr':

    spec




 int [1:392] 0 1 1 1 1 1 1 0 1 0 ...
 int [1:392] 0 0 0 1 1 1 0 0 0 0 ...


In [14]:
cf <- conf_mat(df1, truth = obs, estimate = pred)
summary(cf)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7678571
kap,binary,0.4405621
sens,binary,0.7898305
spec,binary,0.7010309
ppv,binary,0.8893130
npv,binary,0.5230769
mcc,binary,0.4499181
j_index,binary,0.4908614
bal_accuracy,binary,0.7454307
